In [32]:
import torch
import pickle
import numpy as np
import time
from matplotlib import pyplot as plt
import multiprocessing

In [2]:
folder = 'data/pkl/'
min_gps = np.array([104.042102,30.652828])
max_gps = np.array([104.129591,30.727818])

In [3]:
grid_data = pickle.load(open(folder + 'hex_grid.pkl', 'rb'))
'''
gx = np.array(grid_data['vertex'][:,:,0])
gy = np.array(grid_data['vertex'][:,:,1])

grid_data['vertex'][:,:,0] = gy
grid_data['vertex'][:,:,1] = gx

pickle.dump(grid_data, open(folder + 'hex_grid.pkl', 'wb'))
'''
'''
grid_mean = np.array(grid_data['vertex'])
grid_mean_x = grid_mean[:,:,0]
grid_mean_y = grid_mean[:,:,1]
grid_mean_x[grid_mean_x < min_gps[0]] = min_gps[0]
grid_mean_y[grid_mean_y < min_gps[1]] = min_gps[1]
grid_mean_x[grid_mean_x > max_gps[0]] = max_gps[0]
grid_mean_y[grid_mean_y > max_gps[1]] = max_gps[1]
print(grid_mean[:,:,0].min(),grid_mean[:,:,0].max())
print(grid_mean[:,:,1].min(),grid_mean[:,:,1].max())
grid_mean = grid_mean.mean(axis=1)
grid_data['mean'] = grid_mean
'''
None

In [59]:
ride_req_data = pickle.load(open(folder + 'ride_req.pkl', 'rb'))

In [5]:
def isRayIntersectsSegment(poi,s_poi,e_poi): #[x,y] [lng,lat]
    #输入：判断点，边起点，边终点，都是[lng,lat]格式数组
    if s_poi[1]==e_poi[1]: #排除与射线平行、重合，线段首尾端点重合的情况
        return False
    if s_poi[1]>poi[1] and e_poi[1]>poi[1]: #线段在射线上边
        return False
    if s_poi[1]<poi[1] and e_poi[1]<poi[1]: #线段在射线下边
        return False
    if s_poi[1]==poi[1] and e_poi[1]>poi[1]: #交点为下端点，对应spoint
        return False
    if e_poi[1]==poi[1] and s_poi[1]>poi[1]: #交点为下端点，对应epoint
        return False
    if s_poi[0]<poi[0] and e_poi[0]<poi[0]: #线段在射线左边
        return False

    xseg=e_poi[0]-(e_poi[0]-s_poi[0])*(e_poi[1]-poi[1])/(e_poi[1]-s_poi[1]) #求交
    if xseg<poi[0]: #交点在射线起点的左侧
        return False
    return True  #排除上述情况之后

def isPoiWithinPoly(poi,poly):
    sinsc=0 #交点个数
    poly = [*poly, poly[0]]
    for s_poi, e_poi in zip(poly[:-1], poly[1:]): #[0,len-1]
        if isRayIntersectsSegment(poi,s_poi,e_poi):
            sinsc+=1 #有交点就加1

    return True if sinsc%2==1 else  False

def find_grid_idx(point, grid = grid_data):
    res = -1
    for i in range(len(grid['ID'])):
        if isPoiWithinPoly(point, grid['vertex'][i]):
            if res != -1:
                print(res, i, point)
            res = i
    return res

In [6]:
def extract_time_feat(ts):
    lt = time.localtime(ts)
    return lt.tm_mday, lt.tm_hour, lt.tm_wday, lt.tm_wday >= 5

查看数据分布。存在脏数据，碰到要忽略

In [7]:
ride_req_data['pos'].min(),ride_req_data['pos'].max()
r = ride_req_data['pos']
print(r.shape)
print(np.histogram(r[:,:,0].reshape(-1)))
print(np.histogram(r[:,:,1].reshape(-1)))
print(grid_data['vertex'].min())
print(grid_data['vertex'].max())
v = grid_data['vertex']
print(v.shape)
print(np.histogram(v[:,:,0].reshape(-1)))
print(np.histogram(v[:,:,1].reshape(-1)))

(7065937, 2, 2)
(array([     163,        0,        0,        0,        0,        0,
              0,        0, 14131702,        9]), array([  0.      ,  12.036152,  24.072304,  36.108456,  48.144608,
        60.18076 ,  72.216912,  84.253064,  96.289216, 108.325368,
       120.36152 ]))
(array([     163,        0,        0,        0,        0,        3,
              3, 14131701,        2,        2]), array([ 0.      ,  4.014547,  8.029094, 12.043641, 16.058188, 20.072735,
       24.087282, 28.101829, 32.116376, 36.130923, 40.14547 ]))
22.85281
120.36612
(8518, 6, 2)
(array([50062,   940,    58,     0,     0,    12,     6,     6,    18,
           6]), array([102.99358 , 104.730834, 106.468088, 108.205342, 109.942596,
       111.67985 , 113.417104, 115.154358, 116.891612, 118.628866,
       120.36612 ]))
(array([   18,     6,     6,   131, 50881,    48,     6,     0,     0,
          12]), array([22.85281 , 24.582904, 26.312998, 28.043092, 29.773186, 31.50328 ,
       33.233374, 34.963

多线程查找每个轨迹起点所在区域

In [11]:
threadnum = 29

select_length = len(ride_req_data['ID'])

class req_calc_thread(multiprocessing.Process):
    def __init__ (self, threadID, start, inputdata, sharelist):
        multiprocessing.Process.__init__(self)
        self.ID = threadID
        self.start_pos = start
        self.inputdata = inputdata
        self.sharelist = sharelist
        print('thread %d: item length = %d' % (threadID, len(self.inputdata)))
    
    def run(self):
        start_time = time.time()
        count = 0
        for pos in self.inputdata:
            #t = extract_time_feat(t[0])
            s_pos = find_grid_idx(pos)
            self.sharelist[self.start_pos + count] = s_pos
            if s_pos == -1: print(pos)
            count += 1
            if self.ID == threadnum - 1 and count % 1000 == 0:
                print(self.ID, count, time.time() - start_time)

count = 0
m = {}
req_zipped = []
for i in list(zip(ride_req_data['ID'], ride_req_data['time'], ride_req_data['pos'], ride_req_data['reward'])):
    if i[0] not in m:
        req_zipped.append(i)
        m[i[0]] = 1
        
print(len(req_zipped))

req_zipped = list(zip(ride_req_data['ID'], ride_req_data['time'], ride_req_data['pos'], ride_req_data['reward']))[:select_length]

manager = multiprocessing.Manager()
sharelist = manager.list()
for i in range(len(req_zipped)):
    sharelist.append(0)
    if i % 10000 == 0:
        print('sharelist append %d' % i)

threads = []
pos = ride_req_data['pos'][:select_length,1]
print(pos.shape)
for i in range(threadnum):
    delta = len(pos) // threadnum
    start = delta * i
    end = start + delta
    if i + 1 == threadnum:
        end = len(pos)
    #print(i, start, end)
    threads.append(req_calc_thread(i, start, pos[start:end], sharelist))
    threads[i].start()
for t in threads:
    t.join()
print('end over')
end_grid = list(sharelist)

threads = []
pos = ride_req_data['pos'][:select_length,0]
print(pos.shape)
for i in range(threadnum):
    delta = len(pos) // threadnum
    start = delta * i
    end = start + delta
    if i + 1 == threadnum:
        end = len(pos)
    #print(i, start, end)
    threads.append(req_calc_thread(i, start, pos[start:end], sharelist))
    threads[i].start()
for t in threads:
    t.join()
print('start over')
start_grid = list(sharelist)

ride_req_data['start_grid'] = start_grid
ride_req_data['end_grid'] = end_grid
pickle.dump(ride_req_data, open(folder + 'ride_req.pkl', 'wb'))

6105003
sharelist append 0
(3210, 2)
thread 0: item length = 110
thread 1: item length = 110
thread 2: item length = 110
thread 3: item length = 110
thread 4: item length = 110
thread 5: item length = 110
thread 6: item length = 110
thread 7: item length = 110
thread 8: item length = 110
thread 9: item length = 110
thread 10: item length = 110
thread 11: item length = 110
thread 12: item length = 110
thread 13: item length = 110
thread 14: item length = 110
thread 15: item length = 110
thread 16: item length = 110
thread 17: item length = 110
thread 18: item length = 110
thread 19: item length = 110
thread 20: item length = 110
thread 21: item length = 110
thread 22: item length = 110
thread 23: item length = 110
thread 24: item length = 110
thread 25: item length = 110
thread 26: item length = 110
thread 27: item length = 110
thread 28: item length = 130
end over
(3210, 2)
thread 0: item length = 110
thread 1: item length = 110
thread 2: item length = 110
thread 3: item length = 110
t

In [63]:
ride_req_data.keys()

dict_keys(['pos', 'ID', 'end_grid', 'grid', 'reward', 'time', 'start_grid'])

In [85]:
grid_order_count = np.zeros((len(grid_data['ID']) + 1, 31, 24, 7), dtype='int')
grid_reward_count = np.zeros((len(grid_data['ID']) + 1, 31, 24, 7), dtype='float')

req_zipped = list(zip(ride_req_data['ID'], ride_req_data['start_grid'], ride_req_data['time'], ride_req_data['pos'], ride_req_data['reward']))

m = set()

for num, [i, s_pos, t, pos, re] in enumerate(req_zipped):
    if i in m:
        continue
    m.add(i)
    t = extract_time_feat(t[0])
    grid_order_count[s_pos, t[0], t[1], t[2]] += 1
    grid_reward_count[s_pos, t[0], t[1], t[2]] += re
grid_order_count = grid_order_count[:-1,1:]
grid_reward_count = grid_reward_count[:-1,1:]

In [86]:
print(grid_order_count.shape, grid_reward_count.shape)

(8518, 30, 24, 7) (8518, 30, 24, 7)


In [101]:
grid_order = np.array(grid_order_count).sum(axis=1)
grid_reward = np.array(grid_reward_count).sum(axis=1)
grid_order[grid_order == 0] = 1
grid_reward /= grid_order
grid_order = np.array(grid_order_count).sum(axis=1)
pickle.dump({'order': grid_order, 'reward': grid_reward}, open(folder + 'grid_order_reward.pkl', 'wb'))

In [102]:
print(grid_order.max(), grid_reward.max())
print(grid_order.shape, grid_reward.shape)

3604 93.05
(8518, 24, 7) (8518, 24, 7)


处理取消概率数据

In [113]:
cancel_data = pickle.load(open(folder + 'cancel_prob.pkl', 'rb'))

In [114]:
m = {}
for i, p in zip(cancel_data['ID'], cancel_data['prob']):
    if i in m.keys():
        if (p != m[i]).any():
            print(i, p, m[i])
            break
        continue
    m[i] = p

In [115]:
m = {}
for i, t, pos, re in zip(ride_req_data['ID'], ride_req_data['time'], ride_req_data['pos'], ride_req_data['reward']):
    if i in m.keys():
        if (t != m[i][0]).any() or (pos != m[i][1]).any() or (re != m[i][2]).any():
            print(i, t, pos, re, m[i])
            break
        continue
    m[i] = [t,pos,re]

In [118]:
print(cancel_data['prob'].shape, len(m))

(7065937, 10) 6105003


In [127]:
m = set()
# startG, endG, hour, weekday, isweekend, reward, order ETA
cancel_train = []
L = len(cancel_data['ID'])
for i in range(L):
    ID = cancel_data['ID'][i]
    if ID in m:
        continue
    m.add(ID)
    startG = ride_req_data['start_grid'][i]
    endG = ride_req_data['end_grid'][i]
    t = ride_req_data['time'][i]
    st = extract_time_feat(t[0])
    hour = st[1]
    weekday = st[2]
    isweekend = st[3]
    reward = ride_req_data['reward'][i]
    ETA = t[1] - t[0]
    prob = cancel_data['prob'][i]
    if startG == -1 or endG == -1:
        #print([ride_req_data['pos'][i], endG, hour, weekday, isweekend, reward, ETA, prob])
        continue
    cancel_train.append([startG, endG, hour, weekday, isweekend, reward, ETA, prob])

In [128]:
print(len(cancel_train))
startG, endG, hour, weekday, isweekend, reward, ETA, prob = list(zip(*cancel_train))
GID = np.array(list(zip(startG, endG)))
splittime = np.array(list(zip(hour, weekday, isweekend)))
reward = np.array(reward)
ETA = np.array(ETA)
prob = np.array(prob)

cancel_train = {
    'GID': GID,
    'time': splittime,
    'reward': reward,
    'ETA': ETA,
    'prob': prob
}
pickle.dump(cancel_train, open(folder + 'cancel_prob_train.pkl', 'wb'))

6104887


In [124]:
cancel_train

{'ETA': array([1710, 2090, 1265, ..., 2145, 1545, 1031]),
 'GID': array([[3046, 3944],
        [6400, 5081],
        [6912, 1720],
        ...,
        [5422, 6971],
        [7839,   60],
        [3735, 4176]]),
 'prob': array([[0.008606, 0.008098, 0.008355, ..., 0.042079, 0.043802, 0.046584],
        [0.117864, 0.126627, 0.134129, ..., 0.210413, 0.227839, 0.237643],
        [0.011333, 0.010759, 0.011035, ..., 0.041103, 0.044903, 0.050505],
        ...,
        [0.008664, 0.009315, 0.009673, ..., 0.046172, 0.050188, 0.056687],
        [0.013174, 0.014159, 0.014555, ..., 0.05161 , 0.056076, 0.063198],
        [0.012617, 0.010593, 0.010353, ..., 0.058398, 0.070243, 0.084266]]),
 'reward': array([ 3.54, 11.7 ,  5.01, ...,  5.27,  5.72,  2.38]),
 'time': array([[ 9,  1,  0],
        [15,  1,  0],
        [20,  1,  0],
        ...,
        [20,  2,  0],
        [18,  2,  0],
        [19,  2,  0]])}